# Task 4: Machine Learning & Statistical Modeling

## Objective
Build and evaluate predictive models that form the core of a dynamic, risk-based pricing system.

## Modeling Goals:

1. **Claim Severity Prediction (Risk Model)**: For policies that have a claim, build a model to predict the TotalClaims amount.
   - Target Variable: TotalClaims (on subset where claims > 0)
   - Evaluation Metrics: RMSE, R²

2. **Premium Optimization (Pricing Framework)**: Develop a machine learning model to predict an appropriate premium.
   - Target Variable: CalculatedPremiumPerTerm or TotalPremium
   - Evaluation Metrics: RMSE, R²

3. **Claim Probability Prediction (Advanced)**: Build a model to predict the probability of a claim occurring.
   - Target Variable: Binary indicator (HasClaim)
   - Evaluation Metrics: Accuracy, Precision, Recall, F1, ROC-AUC

4. **Zipcode-Level Linear Regression**: For each zipcode, fit a linear regression model that predicts total claims.

## Models to Implement:
- Linear Regression
- Random Forest
- XGBoost

## Model Interpretability:
- Feature Importance Analysis
- SHAP (SHapley Additive exPlanations) values
- Business interpretation of important features


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from data_processing import load_and_validate_data, engineer_features
from modeling import (
    prepare_features_for_modeling, encode_categorical_features,
    impute_missing_values, train_linear_regression, train_random_forest,
    train_xgboost, get_feature_importance, calculate_shap_values
)
from utils import calculate_claim_frequency, calculate_claim_severity

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")


## 1. Data Loading and Preparation


In [ ]:
# Load the data
data_path = '../data/raw/insurance_data.csv'  # Update this path

try:
    df = load_and_validate_data(data_path)
    print(f"Data loaded successfully!")
    print(f"Shape: {df.shape}")
except FileNotFoundError:
    print(f"Data file not found at {data_path}")
    print("Please update the data_path variable with the correct path to your insurance data.")
    print("\nCreating sample data for demonstration...")
    # Create sample data
    np.random.seed(42)
    n_samples = 10000
    df = pd.DataFrame({
        'PolicyID': range(n_samples),
        'TransactionMonth': pd.date_range('2014-02-01', periods=n_samples, freq='D'),
        'Province': np.random.choice(['Gauteng', 'Western Cape', 'KwaZulu-Natal', 'Eastern Cape'], n_samples),
        'PostalCode': np.random.choice([1000, 2000, 3000, 4000, 5000, 6000], n_samples),
        'Gender': np.random.choice(['Male', 'Female'], n_samples),
        'VehicleType': np.random.choice(['Sedan', 'SUV', 'Hatchback', 'Coupe'], n_samples),
        'Make': np.random.choice(['Toyota', 'Ford', 'BMW', 'Mercedes', 'VW'], n_samples),
        'RegistrationYear': np.random.randint(2000, 2020, n_samples),
        'SumInsured': np.random.uniform(50000, 500000, n_samples),
        'TotalPremium': np.random.uniform(5000, 50000, n_samples),
        'CalculatedPremiumPerTerm': np.random.uniform(4000, 45000, n_samples),
        'TotalClaims': np.random.exponential(10000, n_samples) * (np.random.random(n_samples) < 0.3),
        'CustomValueEstimate': np.random.uniform(50000, 500000, n_samples)
    })
    print("Sample dataframe created for demonstration purposes.")


In [ ]:
# Engineer features
df = engineer_features(df)
print(f"After feature engineering: {df.shape}")
print(f"\nNew features created:")
new_features = ['VehicleAge', 'HasClaim', 'PremiumToSumInsuredRatio', 'LossRatio', 
                'VehicleValueCategory', 'PremiumCategory']
for feat in new_features:
    if feat in df.columns:
        print(f"  - {feat}")


## 2. Model 1: Claim Severity Prediction (Risk Model)

**Objective**: Predict TotalClaims amount for policies that have claims.
**Target**: TotalClaims (on subset where TotalClaims > 0)


In [ ]:
# Prepare data for claim severity prediction (only policies with claims)
df_claims = df[df['TotalClaims'] > 0].copy()
print(f"Policies with claims: {len(df_claims):,} ({len(df_claims)/len(df)*100:.2f}%)")

if len(df_claims) < 100:
    print("⚠ Warning: Very few policies with claims. Model may not perform well.")
else:
    # Prepare features
    X_sev, y_sev, cat_cols_sev, num_cols_sev = prepare_features_for_modeling(
        df_claims, 
        target_col='TotalClaims',
        exclude_cols=['PolicyID', 'UnderwrittenCoverID', 'TotalPremium', 'CalculatedPremiumPerTerm']
    )
    
    # Encode categorical features
    X_sev_encoded, encoder_sev = encode_categorical_features(X_sev, cat_cols_sev, method='onehot')
    
    # Impute missing values
    X_sev_processed, imputers_sev = impute_missing_values(X_sev_encoded)
    
    # Train-test split
    X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(
        X_sev_processed, y_sev, test_size=0.2, random_state=42
    )
    
    print(f"\nTraining set: {len(X_train_sev):,} samples")
    print(f"Test set: {len(X_test_sev):,} samples")
    print(f"Features: {X_train_sev.shape[1]}")


In [ ]:
# Train models for claim severity prediction
severity_models = {}

if len(df_claims) >= 100:
    # Linear Regression
    print("="*80)
    print("Training Linear Regression for Claim Severity...")
    print("="*80)
    severity_models['linear'] = train_linear_regression(
        X_train_sev, y_train_sev, X_test_sev, y_test_sev
    )
    
    # Random Forest
    print("\n" + "="*80)
    print("Training Random Forest for Claim Severity...")
    print("="*80)
    severity_models['rf'] = train_random_forest(
        X_train_sev, y_train_sev, X_test_sev, y_test_sev,
        is_classification=False, n_estimators=100
    )
    
    # XGBoost
    print("\n" + "="*80)
    print("Training XGBoost for Claim Severity...")
    print("="*80)
    severity_models['xgb'] = train_xgboost(
        X_train_sev, y_train_sev, X_test_sev, y_test_sev,
        is_classification=False, n_estimators=100
    )
    
    print("\n" + "="*80)
    print("CLAIM SEVERITY MODEL COMPARISON")
    print("="*80)
    comparison_sev = pd.DataFrame({
        'Model': [m['model_name'] for m in severity_models.values()],
        'Test RMSE': [m['test_rmse'] for m in severity_models.values()],
        'Test R²': [m['test_r2'] for m in severity_models.values()],
        'Test MAE': [m['test_mae'] for m in severity_models.values()]
    })
    print(comparison_sev.to_string(index=False))
else:
    print("Insufficient data for claim severity modeling.")


In [ ]:
# Prepare data for premium prediction
target_premium = 'CalculatedPremiumPerTerm' if 'CalculatedPremiumPerTerm' in df.columns else 'TotalPremium'
print(f"Using target: {target_premium}")

X_prem, y_prem, cat_cols_prem, num_cols_prem = prepare_features_for_modeling(
    df,
    target_col=target_premium,
    exclude_cols=['PolicyID', 'UnderwrittenCoverID', 'TotalClaims']
)

# Encode categorical features
X_prem_encoded, encoder_prem = encode_categorical_features(X_prem, cat_cols_prem, method='onehot')

# Impute missing values
X_prem_processed, imputers_prem = impute_missing_values(X_prem_encoded)

# Train-test split
X_train_prem, X_test_prem, y_train_prem, y_test_prem = train_test_split(
    X_prem_processed, y_prem, test_size=0.2, random_state=42
)

print(f"\nTraining set: {len(X_train_prem):,} samples")
print(f"Test set: {len(X_test_prem):,} samples")
print(f"Features: {X_train_prem.shape[1]}")


In [ ]:
# Train models for premium prediction
premium_models = {}

print("="*80)
print("Training Linear Regression for Premium Prediction...")
print("="*80)
premium_models['linear'] = train_linear_regression(
    X_train_prem, y_train_prem, X_test_prem, y_test_prem
)

print("\n" + "="*80)
print("Training Random Forest for Premium Prediction...")
print("="*80)
premium_models['rf'] = train_random_forest(
    X_train_prem, y_train_prem, X_test_prem, y_test_prem,
    is_classification=False, n_estimators=100
)

print("\n" + "="*80)
print("Training XGBoost for Premium Prediction...")
print("="*80)
premium_models['xgb'] = train_xgboost(
    X_train_prem, y_train_prem, X_test_prem, y_test_prem,
    is_classification=False, n_estimators=100
)

print("\n" + "="*80)
print("PREMIUM PREDICTION MODEL COMPARISON")
print("="*80)
comparison_prem = pd.DataFrame({
    'Model': [m['model_name'] for m in premium_models.values()],
    'Test RMSE': [m['test_rmse'] for m in premium_models.values()],
    'Test R²': [m['test_r2'] for m in premium_models.values()],
    'Test MAE': [m['test_mae'] for m in premium_models.values()]
})
print(comparison_prem.to_string(index=False))


## 4. Model 3: Claim Probability Prediction (Binary Classification)

**Objective**: Predict the probability of a claim occurring.
**Target**: HasClaim (binary: 0 = no claim, 1 = has claim)


In [ ]:
# Prepare data for claim probability prediction (binary classification)
if 'HasClaim' not in df.columns:
    df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)

print(f"Claim frequency: {df['HasClaim'].mean():.4f} ({df['HasClaim'].mean()*100:.2f}%)")
print(f"Policies with claims: {df['HasClaim'].sum():,}")
print(f"Policies without claims: {(df['HasClaim'] == 0).sum():,}")

X_prob, y_prob, cat_cols_prob, num_cols_prob = prepare_features_for_modeling(
    df,
    target_col='HasClaim',
    exclude_cols=['PolicyID', 'UnderwrittenCoverID', 'TotalClaims']
)

# Encode categorical features
X_prob_encoded, encoder_prob = encode_categorical_features(X_prob, cat_cols_prob, method='onehot')

# Impute missing values
X_prob_processed, imputers_prob = impute_missing_values(X_prob_encoded)

# Train-test split
X_train_prob, X_test_prob, y_train_prob, y_test_prob = train_test_split(
    X_prob_processed, y_prob, test_size=0.2, random_state=42, stratify=y_prob
)

print(f"\nTraining set: {len(X_train_prob):,} samples")
print(f"Test set: {len(X_test_prob):,} samples")
print(f"Features: {X_train_prob.shape[1]}")


In [ ]:
# Train models for claim probability prediction
probability_models = {}

print("="*80)
print("Training Random Forest for Claim Probability...")
print("="*80)
probability_models['rf'] = train_random_forest(
    X_train_prob, y_train_prob, X_test_prob, y_test_prob,
    is_classification=True, n_estimators=100
)

print("\n" + "="*80)
print("Training XGBoost for Claim Probability...")
print("="*80)
probability_models['xgb'] = train_xgboost(
    X_train_prob, y_train_prob, X_test_prob, y_test_prob,
    is_classification=True, n_estimators=100
)

print("\n" + "="*80)
print("CLAIM PROBABILITY MODEL COMPARISON")
print("="*80)
comparison_prob = pd.DataFrame({
    'Model': [m['model_name'] for m in probability_models.values()],
    'Test Accuracy': [m['test_accuracy'] for m in probability_models.values()],
    'Test Precision': [m['test_precision'] for m in probability_models.values()],
    'Test Recall': [m['test_recall'] for m in probability_models.values()],
    'Test F1': [m['test_f1'] for m in probability_models.values()]
})
if 'test_roc_auc' in probability_models['rf']:
    comparison_prob['Test ROC-AUC'] = [m.get('test_roc_auc', np.nan) for m in probability_models.values()]
print(comparison_prob.to_string(index=False))


## 5. Model 4: Zipcode-Level Linear Regression

**Objective**: For each zipcode, fit a linear regression model that predicts total claims.


In [ ]:
# Zipcode-level linear regression
if 'PostalCode' in df.columns:
    zipcode_models = {}
    zipcode_results = []
    
    # Get zipcodes with sufficient data
    zipcode_counts = df['PostalCode'].value_counts()
    valid_zipcodes = zipcode_counts[zipcode_counts >= 50].index.tolist()
    
    print(f"Fitting linear regression models for {len(valid_zipcodes)} zipcodes...")
    print(f"(Minimum 50 samples per zipcode required)\n")
    
    for zipcode in valid_zipcodes[:20]:  # Limit to top 20 for performance
        zipcode_data = df[df['PostalCode'] == zipcode].copy()
        
        if len(zipcode_data) < 50:
            continue
        
        # Prepare features (aggregate by zipcode)
        features = ['TotalPremium', 'SumInsured']
        if 'CustomValueEstimate' in zipcode_data.columns:
            features.append('CustomValueEstimate')
        
        X_zip = zipcode_data[features].values
        y_zip = zipcode_data['TotalClaims'].values
        
        # Train linear regression
        try:
            model_zip = LinearRegression()
            model_zip.fit(X_zip, y_zip)
            
            # Predictions
            y_pred_zip = model_zip.predict(X_zip)
            
            # Metrics
            rmse = np.sqrt(mean_squared_error(y_zip, y_pred_zip))
            r2 = r2_score(y_zip, y_pred_zip)
            
            zipcode_models[zipcode] = model_zip
            zipcode_results.append({
                'PostalCode': zipcode,
                'SampleSize': len(zipcode_data),
                'RMSE': rmse,
                'R²': r2,
                'Coefficients': model_zip.coef_.tolist()
            })
        except Exception as e:
            print(f"Error fitting model for zipcode {zipcode}: {str(e)}")
    
    zipcode_results_df = pd.DataFrame(zipcode_results)
    print("\nZipcode-Level Linear Regression Results:")
    print(zipcode_results_df.to_string(index=False))
else:
    print("PostalCode column not found in dataset.")


In [ ]:
# Feature importance for best premium prediction model
best_prem_model_name = comparison_prem.loc[comparison_prem['Test R²'].idxmax(), 'Model']
best_prem_model_key = [k for k, v in premium_models.items() if v['model_name'] == best_prem_model_name][0]
best_prem_model = premium_models[best_prem_model_key]['model']

print(f"Analyzing feature importance for: {best_prem_model_name}")
print("="*80)

# Get feature importance
feature_importance = get_feature_importance(best_prem_model, X_train_prem.columns.tolist(), top_n=15)

if not feature_importance.empty:
    print("\nTop 15 Most Important Features:")
    print(feature_importance.to_string(index=False))
    
    # Visualize feature importance
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.barh(range(len(feature_importance)), feature_importance['importance'].values, edgecolor='black', alpha=0.7)
    ax.set_yticks(range(len(feature_importance)))
    ax.set_yticklabels(feature_importance['feature'].values)
    ax.set_xlabel('Feature Importance', fontweight='bold')
    ax.set_title(f'Feature Importance - {best_prem_model_name}', fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig('../figures/feature_importance_premium.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Could not extract feature importance.")


In [ ]:
# SHAP Analysis for model interpretability
try:
    print("="*80)
    print("SHAP Analysis for Model Interpretability")
    print("="*80)
    
    # Use best premium model
    shap_results = calculate_shap_values(
        best_prem_model,
        X_test_prem.sample(min(100, len(X_test_prem))),
        max_samples=100
    )
    
    if shap_results:
        print("\nTop Features by Mean Absolute SHAP Value:")
        print(shap_results['feature_importance'].head(15).to_string(index=False))
        
        # Visualize SHAP summary
        try:
            import shap
            fig, ax = plt.subplots(figsize=(12, 8))
            shap.summary_plot(
                shap_results['shap_values'],
                shap_results['X_sample'],
                plot_type="bar",
                max_display=15,
                show=False
            )
            plt.tight_layout()
            plt.savefig('../figures/shap_summary.png', dpi=300, bbox_inches='tight')
            plt.show()
        except Exception as e:
            print(f"Could not create SHAP plot: {str(e)}")
    else:
        print("SHAP analysis not available.")
        
except Exception as e:
    print(f"Error in SHAP analysis: {str(e)}")
    print("Make sure SHAP is installed: pip install shap")


## 7. Model Interpretability & Business Insights

Interpret the top features and their business implications.


In [ ]:
# Business interpretation of feature importance
print("="*80)
print("BUSINESS INTERPRETATION OF MODEL FEATURES")
print("="*80)

if not feature_importance.empty:
    top_features = feature_importance.head(10)
    
    print("\nTop 10 Features Influencing Premium Prediction:\n")
    for idx, row in top_features.iterrows():
        feature = row['feature']
        importance = row['importance']
        
        print(f"{idx+1}. {feature} (Importance: {importance:.4f})")
        
        # Provide business interpretation
        if 'VehicleAge' in feature or 'RegistrationYear' in feature:
            print("   → Older vehicles may require higher premiums due to increased risk")
        elif 'Province' in feature:
            print("   → Geographic location significantly impacts premium pricing")
        elif 'VehicleType' in feature or 'Make' in feature:
            print("   → Vehicle characteristics are key drivers of insurance costs")
        elif 'SumInsured' in feature or 'CustomValueEstimate' in feature:
            print("   → Vehicle value directly influences premium calculations")
        elif 'Gender' in feature:
            print("   → Gender may be a factor (ensure regulatory compliance)")
        elif 'PostalCode' in feature:
            print("   → Location granularity affects risk assessment")
        print()
    
    print("\nKey Business Recommendations:")
    print("1. Focus pricing strategy on top 5-10 features identified by the model")
    print("2. Consider risk-based adjustments for high-importance features")
    print("3. Monitor feature importance over time as market conditions change")
    print("4. Use SHAP values to explain individual premium predictions to customers")
    print("5. Validate model predictions against business rules and regulatory requirements")


## 8. Summary and Model Comparison

Compare all models and provide final recommendations.


In [ ]:
# Final summary
print("="*100)
print("MODELING SUMMARY REPORT")
print("="*100)

print("\n1. CLAIM SEVERITY PREDICTION (Risk Model)")
print("-" * 100)
if severity_models:
    best_sev = max(severity_models.items(), key=lambda x: x[1]['test_r2'])
    print(f"Best Model: {best_sev[1]['model_name']}")
    print(f"  Test RMSE: {best_sev[1]['test_rmse']:.2f}")
    print(f"  Test R²: {best_sev[1]['test_r2']:.4f}")
    print(f"  Test MAE: {best_sev[1]['test_mae']:.2f}")
else:
    print("Models not trained (insufficient data)")

print("\n2. PREMIUM OPTIMIZATION (Pricing Framework)")
print("-" * 100)
if premium_models:
    best_prem = max(premium_models.items(), key=lambda x: x[1]['test_r2'])
    print(f"Best Model: {best_prem[1]['model_name']}")
    print(f"  Test RMSE: {best_prem[1]['test_rmse']:.2f}")
    print(f"  Test R²: {best_prem[1]['test_r2']:.4f}")
    print(f"  Test MAE: {best_prem[1]['test_mae']:.2f}")

print("\n3. CLAIM PROBABILITY PREDICTION (Binary Classification)")
print("-" * 100)
if probability_models:
    best_prob = max(probability_models.items(), key=lambda x: x[1]['test_f1'])
    print(f"Best Model: {best_prob[1]['model_name']}")
    print(f"  Test Accuracy: {best_prob[1]['test_accuracy']:.4f}")
    print(f"  Test F1 Score: {best_prob[1]['test_f1']:.4f}")
    if 'test_roc_auc' in best_prob[1]:
        print(f"  Test ROC-AUC: {best_prob[1]['test_roc_auc']:.4f}")

print("\n4. ZIPCODE-LEVEL LINEAR REGRESSION")
print("-" * 100)
if 'zipcode_results_df' in locals() and not zipcode_results_df.empty:
    avg_r2 = zipcode_results_df['R²'].mean()
    print(f"Average R² across zipcodes: {avg_r2:.4f}")
    print(f"Number of zipcodes modeled: {len(zipcode_results_df)}")

print("\n" + "="*100)
print("MODELING COMPLETE!")
print("="*100)
